In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 32.0 MB/s eta 0:00:00


In [ ]:
from pulp import *
import math
import copy
import random

In [ ]:
def GenerateGrid(n):
  global grid
  grid = [[0] * n**2 for i in range(n**2)]

  size=len(grid)
  bsize=int(math.sqrt(len(grid)))
  index_list=[]
  for i in range(1,size+1):
      list = []
      for j in range(1,size+1):
          index = (i,j)
          list.append(index)
      index_list.append(list)
  box_list=[]
  for i in range(0,bsize):
      for j in range(0,bsize):
          box = [index_list[bsize*i+p][bsize*j+q] for p in range(0,bsize) for q in range(0,bsize)]
          box_list.append(box)

  lp = LpProblem("Sudoku LP",LpMinimize)
  rows = range(1,size+1)
  cols = range(1,size+1)
  nums = range(1,size+1)

  x_ijk = LpVariable.dicts("x",(rows,cols,nums),0,1,LpInteger)
  obj= lpSum(x_ijk[i][j][k] for i in rows for j in cols for k in nums)
  lp += obj, "Objective Function"

  for j in cols:
      for k in nums:
          lp += lpSum([x_ijk[i][j][k] for i in rows]) == 1,""
  for i in rows:
      for k in nums:
          lp += lpSum([x_ijk[i][j][k] for j in cols]) == 1,""
  for i in rows:
      for j in cols:
          lp += lpSum([x_ijk[i][j][k] for k in nums]) == 1,""
  for k in nums:
      for box in box_list:
          lp += lpSum([x_ijk[i][j][k] for (i,j) in box]) == 1,""
  for i in range(len(grid)):
      for j in range(len(grid)):
          if grid[i][j]==0:
            continue
          else:
            lp += x_ijk[i+1][j+1][grid[i][j]] == 1,""

  for a in range(1,random.randint(1, n)+1):
    numbers = [*range(1,size+1)]
    random.shuffle(numbers)
    lp.solve()
    for i in range(len(grid)):
      for j in range(len(grid)):
        for k in range(len(grid)):
          if x_ijk[i+1][j+1][k+1].varValue==1:
            grid[i][j] = numbers[k]
          else:
            continue
    lp += lpSum(x_ijk[i][j][k] for k in nums for i in rows for j in cols if value(x_ijk[i][j][k]) == 1) <= (len(grid)**2-1)

  print("Solusi: ")
  al = "╔"; am = "╦"; ar = "╗"; g = "═"
  ml = "╠"; mm = "╬"; mr = "╣"
  bl = "╚"; bm = "╩"; br = "╝"
  pgaris = (bsize*2+1)*g
  for i in rows:
    line=""
    if i==1: print(al+(bsize-1)*(pgaris+am)+(pgaris+ar))
    elif i in range(1+bsize,size,bsize): print(ml+(bsize-1)*(pgaris+mm)+(pgaris+mr))
    for j in cols:
      p = grid[i-1][j-1]
      if j in range(1,size,bsize): line+="║ "
      line += str(p)+" "
      if j == size: line+="║ "
    print(line)
  print(bl+(bsize-1)*(pgaris+bm)+(pgaris+br))


In [ ]:
GenerateGrid(2)

Solusi: 
╔═════╦═════╗
║ 2 1 ║ 3 4 ║ 
║ 4 3 ║ 2 1 ║ 
╠═════╬═════╣
║ 3 4 ║ 1 2 ║ 
║ 1 2 ║ 4 3 ║ 
╚═════╩═════╝


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
def UniqueTest(grid):
    global feasible_solution_count
    size=len(grid)
    bsize=int(math.sqrt(len(grid)))
    index_list=[]
    for i in range(1,size+1):
        list = []
        for j in range(1,size+1):
            index = (i,j)
            list.append(index)
        index_list.append(list)
    box_list=[]
    for i in range(0,bsize):
        for j in range(0,bsize):
            box = [index_list[bsize*i+p][bsize*j+q] for p in range(0,bsize) for q in range(0,bsize)]
            box_list.append(box)

    lp = LpProblem("Sudoku LP",LpMinimize)
    rows = range(1,size+1)
    cols = range(1,size+1)
    nums = range(1,size+1)

    x_ijk = LpVariable.dicts("x",(rows,cols,nums),0,1,LpInteger)
    obj=lpSum(x_ijk[i][j][k] for i in rows for j in cols for k in nums)
    lp += obj, "Objective Function"

    for j in cols:
        for k in nums:
            lp += lpSum([x_ijk[i][j][k] for i in rows]) == 1,""
    for i in rows:
        for k in nums:
            lp += lpSum([x_ijk[i][j][k] for j in cols]) == 1,""
    for i in rows:
        for j in cols:
            lp += lpSum([x_ijk[i][j][k] for k in nums]) == 1,""
    for j in nums:
        for box in box_list:
            lp += lpSum([x_ijk[i][j][k] for (i,j) in box]) == 1,""
    for i in range(len(grid)):
        for j in range(len(grid)):
            if grid[i][j]==0:
              continue
            else:
              lp += x_ijk[i+1][j+1][grid[i][j]] == 1,""
    feasible_solution_count = 0
    while True:
      lp.solve()
      if str(LpStatus[lp.status]) == "Infeasible":
        break
      else:
        lp += lpSum(x_ijk[i][j][k] for k in nums for i in rows for j in cols if value(x_ijk[i][j][k]) == 1) <= (len(grid)**2-1)
        feasible_solution_count += 1
        if feasible_solution_count == 3:
          break

In [ ]:
def GeneratePuzzle(n):
  global newgrid
  global step_list
  global tempgrid1

  GenerateGrid(n)

  size=len(grid)
  bsize=int(math.sqrt(len(grid)))
  rows = range(1,size+1)
  cols = range(1,size+1)
  nums = range(1,size+1)

  newgrid = copy.deepcopy(grid)
  listofindex = []
  for i in range(len(grid)):
    for j in range(len(grid)):
      index = (i,j)
      listofindex.append(index)
  random.shuffle(listofindex)
  x = 0
  step_list = []

  while x != len(listofindex):
    i,j = listofindex[x]
    tempgrid = copy.deepcopy(newgrid)
    tempgrid1 = copy.deepcopy(newgrid)
    tempgrid1[i][j] = 0
    UniqueTest(tempgrid1)
    x += 1
    if feasible_solution_count==1:
      step_list.append(tempgrid1)
      newgrid = copy.deepcopy(tempgrid1)
    else:
      newgrid = copy.deepcopy(tempgrid)
      continue

  print("Masalah: ")
  al = "╔"; am = "╦"; ar = "╗"; g = "═"
  ml = "╠"; mm = "╬"; mr = "╣"
  bl = "╚"; bm = "╩"; br = "╝"
  pgaris = (bsize*2+1)*g
  for i in rows:
    line=""
    if i==1: print(al+(bsize-1)*(pgaris+am)+(pgaris+ar))
    elif i in range(1+bsize,size,bsize): print(ml+(bsize-1)*(pgaris+mm)+(pgaris+mr))
    for j in cols:
      p = newgrid[i-1][j-1]
      if j in range(1,size,bsize): line+="║ "
      line += str(p)+" "
      if j == size: line+="║ "
    print(line)
  print(bl+(bsize-1)*(pgaris+bm)+(pgaris+br))

In [ ]:
print(GeneratePuzzle(2))

Solusi: 
╔═════╦═════╗
║ 4 3 ║ 1 2 ║ 
║ 2 1 ║ 3 4 ║ 
╠═════╬═════╣
║ 1 2 ║ 4 3 ║ 
║ 3 4 ║ 2 1 ║ 
╚═════╩═════╝
Masalah: 
╔═════╦═════╗
║ 4 0 ║ 0 0 ║ 
║ 0 1 ║ 0 0 ║ 
╠═════╬═════╣
║ 0 2 ║ 4 0 ║ 
║ 0 0 ║ 2 1 ║ 
╚═════╩═════╝
None


In [ ]:
print(GeneratePuzzle(3))

Solusi: 
╔═══════╦═══════╦═══════╗
║ 1 6 7 ║ 8 2 9 ║ 3 4 5 ║ 
║ 9 8 3 ║ 4 6 5 ║ 7 1 2 ║ 
║ 4 2 5 ║ 1 3 7 ║ 9 8 6 ║ 
╠═══════╬═══════╬═══════╣
║ 3 5 8 ║ 7 1 4 ║ 2 6 9 ║ 
║ 7 1 2 ║ 9 5 6 ║ 4 3 8 ║ 
║ 6 4 9 ║ 2 8 3 ║ 5 7 1 ║ 
╠═══════╬═══════╬═══════╣
║ 2 3 6 ║ 5 4 1 ║ 8 9 7 ║ 
║ 5 9 1 ║ 3 7 8 ║ 6 2 4 ║ 
║ 8 7 4 ║ 6 9 2 ║ 1 5 3 ║ 
╚═══════╩═══════╩═══════╝
Masalah: 
╔═══════╦═══════╦═══════╗
║ 0 0 7 ║ 8 0 0 ║ 0 0 0 ║ 
║ 0 0 0 ║ 4 6 5 ║ 0 0 0 ║ 
║ 0 2 5 ║ 0 0 0 ║ 9 0 0 ║ 
╠═══════╬═══════╬═══════╣
║ 3 0 8 ║ 7 0 0 ║ 2 0 0 ║ 
║ 0 1 2 ║ 0 5 6 ║ 0 3 0 ║ 
║ 0 0 0 ║ 0 0 0 ║ 0 7 1 ║ 
╠═══════╬═══════╬═══════╣
║ 0 3 0 ║ 0 4 0 ║ 0 0 7 ║ 
║ 0 9 0 ║ 3 7 0 ║ 6 0 0 ║ 
║ 8 0 4 ║ 0 9 0 ║ 0 5 3 ║ 
╚═══════╩═══════╩═══════╝
None


In [ ]:
print(GeneratePuzzle(4))

Solusi: 
╔═════════╦═════════╦═════════╦═════════╗
║ 16 13 12 7 ║ 1 14 5 8 ║ 6 9 3 15 ║ 10 4 2 11 ║ 
║ 2 4 11 6 ║ 16 9 7 3 ║ 12 8 10 1 ║ 13 5 15 14 ║ 
║ 8 5 15 3 ║ 12 10 13 11 ║ 16 14 2 4 ║ 9 1 6 7 ║ 
║ 14 9 10 1 ║ 15 2 4 6 ║ 5 7 13 11 ║ 8 16 3 12 ║ 
╠═════════╬═════════╬═════════╬═════════╣
║ 9 2 16 5 ║ 13 8 14 12 ║ 10 3 15 6 ║ 1 7 11 4 ║ 
║ 7 3 4 8 ║ 10 11 16 15 ║ 9 1 12 5 ║ 2 6 14 13 ║ 
║ 13 14 6 15 ║ 7 5 1 2 ║ 4 11 16 8 ║ 3 10 12 9 ║ 
║ 10 11 1 12 ║ 4 6 3 9 ║ 14 13 7 2 ║ 15 8 16 5 ║ 
╠═════════╬═════════╬═════════╬═════════╣
║ 11 15 3 16 ║ 9 12 10 7 ║ 8 5 4 14 ║ 6 13 1 2 ║ 
║ 5 6 13 10 ║ 8 15 11 1 ║ 3 2 9 7 ║ 12 14 4 16 ║ 
║ 4 7 2 9 ║ 14 3 6 16 ║ 15 12 1 13 ║ 5 11 10 8 ║ 
║ 12 1 8 14 ║ 5 4 2 13 ║ 11 16 6 10 ║ 7 3 9 15 ║ 
╠═════════╬═════════╬═════════╬═════════╣
║ 6 8 7 4 ║ 3 13 15 14 ║ 2 10 11 9 ║ 16 12 5 1 ║ 
║ 15 16 5 11 ║ 6 7 9 10 ║ 1 4 8 12 ║ 14 2 13 3 ║ 
║ 3 12 9 2 ║ 11 1 8 5 ║ 13 6 14 16 ║ 4 15 7 10 ║ 
║ 1 10 14 13 ║ 2 16 12 4 ║ 7 15 5 3 ║ 11 9 8 6 ║ 
╚═════════╩═════════╩══

KeyboardInterrupt: 